## MIC Demo 2 - Parallelising multiple measurements

As the first part of the demonstration 1 shows, obtaining a multivariate MIC measurement requires multiple runs of the MIC algorithm. This is because:
- A multivariate measurement on $K$ variables is  decomposed into a sum of $K$ univariate runs, where each variable is suitably conditioned on the other contemporaneous variables.
- In order to improve accuracy, multiple measurements should be carried out using different conditioning orders for the decomposition and then the average should be taken.
- Finally, because the purpose of the MIC is model comparison, this will need to be carried out for multiple models.

As a result, the obvious solution to this is to parallelise the runs. This can be done using the `multiprocessing` package, as described below, on the same dataset as used in the Demo 1 file.


In [1]:
import os
import sys
import time
import numpy as np
import mic.toolbox as mt
import multiprocessing as mp

import wrapper

Prior to running the multivariate MIC, the steps required for a single run are bundled into a simple wrapper function `wrapper.py`. Note, when running in python from the console, this function can be in the same file as the multiprocessing call below, however, in Jupyter the function [has to be imported to work properly](https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac).

As can be noted from the prinout below, the function contains the same steps as outlined in the Demo 1 file, the only additions being the unpacking/packing of inputs/outputs and the logging of individual tasks to a file.

In [2]:
print(open('wrapper.py').read())

import os
import sys
import time
import numpy as np
import mic.toolbox as mt
import multiprocessing as mp

def MIC_wrapper(inputs):
    """ wrapper function"""

    tic = time.time()

    # Unpack inputs and parameters
    params = inputs[0]
    model = inputs[1]
    var_vec = inputs[2]
    num = inputs[3]
    
    log_path = params['log_path']
    data_path = params['data_path']
    lb = params['lb']
    ub = params['ub']
    r_vec = params['r_vec']
    hp_bit_vec = params['hp_bit_vec']
    mem = params['mem']
    d = params['d']
    lags = params['lags']
    
    print (' Task number {:2d} initialised'.format(num))

    # Redirect output to file and print task/process information
    main_stdout = sys.stdout
    sys.stdout = open(log_path + '//log_' + str(num) + '.out', "w")
    print (' Task number :   {:3d}'.format(num))
    print (' Parent process: {:10d}'.format(os.getppid()))
    print (' Process id:     {:10d}'.format(os.getpid()))    
    
    # Load simulates/empirical data
 

Once this wrapper file is written we can use the `multiprocessing` package to run the same steps on different simulated data and conditioning orders. Note, we are re-using the discretisation settings we used in the Demo 1 file. In a more general application, one would need to make sure that the correct discretisation and memory settings have been found via a small scale exploratory run prior to launching the full analysis.

We create a directory in order to be able to record the output of individual runs:

In [3]:
# Create a directory for saving logs of individual tasks
log_path = 'logs'
if not os.path.exists(log_path):
    os.makedirs(log_path,mode=0o777)

We can now setup the parallel run. We start by setting the common parameters for the run. These are:
- The location of the empirical data and logging folder.
- The discretisation settings `lb`, `ub`, `r_vec` and `hp_bit_vec`.
- The tree and memory settings `mem`, `d` and `lags`.

We then create a list of tuples parametrising each run. In this case, these contain:
- The conditioning order needed for the measurements
- The simulated datasets corresponding to each model

In [4]:
# Set common parameters
params = dict(log_path = log_path,
            data_path = 'data/emp_data.txt',
            lb = [-10,-10],
            ub = [ 10, 10],
            r_vec = [7,7],
            hp_bit_vec = [3,3],
            mem = 200000,
            d = 24,
            lags = 2)

# List all the possible conditioning orders and datasets 
var_vecs = [[1,2],
           [2],
           [2,1],
           [1]]

models = ['data/model_1.txt',
          'data/model_2.txt']

# Populate job (a list) using tuples.
job_inputs = []
count = 1
for model in models:
    for var_vec in var_vecs:
        job_inputs.append((params, model, var_vec, count))
        count += 1
        
num_tasks = len(job_inputs)

Finally, we can use a multiprocessing pool to run the list of jobs in parallel. This simple example can be run on a simple quad core processor, however it can easily be extended to more cores for a larger number of variables $K$ and/or models. 

In [5]:
tic = time.time()
if __name__ == '__main__':
    
    # Set cores - assuming a standard quadcore PC here. 
    num_cores = 4

    # Create pool and run parallel job
    pool = mp.Pool(processes=num_cores)
    results = pool.map(wrapper.MIC_wrapper,job_inputs)

    # Close pool when job is done
    pool.close()

print('Parallel job complete')
print('Elapsed time - {:10.4f} secs.'.format(time.time() - tic))

Parallel job complete
Elapsed time -   369.5989 secs.


Once the parallel run is over, the measurements can be added and averaged to generate the MIC score for each model on each replication. For each model $j$ (1 or 2) the MIC measurement is the average of the two possible ways of decomposing the cross-entropy of a bi-variate system:

$$ \lambda^j (X) = \frac{1}{2}\sum _{t=L}^T \left[ \lambda^j (x_t^1 \mid x_t^2, \Omega_t) + \lambda^j (x_t^2 \mid \Omega_t) \right] + \frac{1}{2}\sum _{t=L}^T \left[ \lambda^j (x_t^2 \mid x_t^1, \Omega_t) + \lambda^j (x_t^1 \mid \Omega_t) \right]$$

In [6]:
mic = np.zeros([2,998,10])
mic_diff = np.zeros([998,10])
task = 0

# Add and average measurements for the two models
for model in range(2):
    for cond_order in range(4):
        mic[model,:,:] += 0.5*results[task]
        task += 1

# Calculate the difference across models, including t-statistic
mic_diff[:,:] = mic[1,:,:] - mic[0,:,:]
mic_diff_v = np.var(mic_diff,0,keepdims = False)
t_stat = np.sum(mic_diff,0)/(mic_diff_v**0.5)

# Print the results
flt_str = '    {:7.2f}'*10    
print('\n MIC scores obtained for Model 1:')
print(flt_str.format(*np.sum(mic[0,:,:],0)))
print('\n MIC scores obtained for Model 2:')
print(flt_str.format(*np.sum(mic[1,:,:],0)))
print('\n\n Difference in MIC (Model 2 - Model 1):')
print(flt_str.format(*np.sum(mic_diff,0)))
print('\n Standard deviation (Model 2 - Model 1):')
print(flt_str.format(*(mic_diff_v**0.5)))
print('\n T statistic (Model 2 - Model 1):')
print(flt_str.format(*(t_stat)))


 MIC scores obtained for Model 1:
    9649.43    9698.55    9663.38    9637.58    9644.74    9677.28    9669.60    9679.73    9685.27    9608.37

 MIC scores obtained for Model 2:
    9582.25    9651.00    9629.27    9620.19    9627.32    9617.76    9636.36    9634.48    9641.34    9544.12


 Difference in MIC (Model 2 - Model 1):
     -67.18     -47.56     -34.11     -17.39     -17.41     -59.51     -33.23     -45.25     -43.93     -64.26

 Standard deviation (Model 2 - Model 1):
       0.57       0.53       0.55       0.53       0.54       0.53       0.51       0.52       0.56       0.48

 T statistic (Model 2 - Model 1):
    -116.97     -90.36     -61.90     -33.00     -32.11    -111.94     -65.13     -86.32     -78.89    -133.56


We would conclude from this exercise that model 2 is the better model as it has a lower MIC score than model 1, and the mean difference seems significant across repilications.

Note that the T-statistic for the difference in means is used here to illustrate that statistical testing is feasilbe as the MIC is provided at the observation level. This assumes that the distribution is normal, however, which is not the case. It is therefore preferable in general to use bootstrap based methods such as the [Model Confidence Set of Hansen et al. 2011](https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA5771).